In [123]:
import pandas as pd
import numpy as np
from utils import *


import seaborn as sns ;sns.set(rc={'figure.figsize':(20,10)})

import matplotlib.pyplot as plt
plt.rcParams ['figure.figsize']  = 25, 10
plt.rcParams ['axes.labelsize']  = 14
plt.rcParams ['xtick.labelsize'] = 12
plt.rcParams ['ytick.labelsize'] = 12
plt.rcParams ['text.color']      = 'black'
plt.rcParams ['axes.labelcolor'] = 'black'
plt.rcParams ['xtick.color']     = 'black'
plt.rcParams ['ytick.color']     = 'black'
plt.style.use('fivethirtyeight')


import warnings
warnings.filterwarnings("ignore")


pd.options.display.float_format = '{:,.5f}'.format
pd.options.display.max_rows = 150
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 0


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Explore datasets

In [124]:
kidney_dataset = pd.read_csv('../data/kidney_disease/kidney_disease.csv')

In [125]:
kidney_dataset.sample(5)

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
158,158,71.00000,60.00000,1.02000,3.00000,2.00000,normal,normal,present,notpresent,424.00000,48.00000,1.50000,132.00000,4.00000,10.90000,31,NaN,NaN,yes,yes,yes,good,no,no,ckd
169,169,nan,70.00000,1.01000,0.00000,2.00000,NaN,normal,notpresent,notpresent,220.00000,68.00000,2.80000,nan,nan,8.70000,27,NaN,NaN,yes,yes,no,good,no,yes,ckd
395,395,55.00000,80.00000,1.02000,0.00000,0.00000,normal,normal,notpresent,notpresent,140.00000,49.00000,0.50000,150.00000,4.90000,15.70000,47,6700,4.9,no,no,no,good,no,no,notckd
289,289,42.00000,70.00000,1.02000,0.00000,0.00000,normal,normal,notpresent,notpresent,93.00000,32.00000,0.90000,143.00000,4.70000,16.60000,43,7100,5.3,no,no,no,good,no,no,notckd
238,238,72.00000,100.00000,nan,nan,nan,NaN,NaN,notpresent,notpresent,201.00000,241.00000,13.40000,127.00000,4.80000,9.40000,28,NaN,NaN,yes,yes,no,good,no,yes,ckd


In [126]:
kidney_dataset.describe()

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
count,400.00000,391.00000,388.00000,353.00000,354.00000,351.00000,356.00000,381.00000,383.00000,313.00000,312.00000,348.00000
mean,199.50000,51.48338,76.46907,1.01741,1.01695,0.45014,148.03652,57.42572,3.07245,137.52875,4.62724,12.52644
std,115.61430,17.16971,13.68364,0.00572,1.35268,1.09919,79.28171,50.50301,5.74113,10.40875,3.19390,2.91259
min,0.00000,2.00000,50.00000,1.00500,0.00000,0.00000,22.00000,1.50000,0.40000,4.50000,2.50000,3.10000
25%,99.75000,42.00000,70.00000,1.01000,0.00000,0.00000,99.00000,27.00000,0.90000,135.00000,3.80000,10.30000
50%,199.50000,55.00000,80.00000,1.02000,0.00000,0.00000,121.00000,42.00000,1.30000,138.00000,4.40000,12.65000
75%,299.25000,64.50000,80.00000,1.02000,2.00000,0.00000,163.00000,66.00000,2.80000,142.00000,4.90000,15.00000
max,399.00000,90.00000,180.00000,1.02500,5.00000,5.00000,490.00000,391.00000,76.00000,163.00000,47.00000,17.80000


In [127]:
clean_noisy_data(kidney_dataset).sample(5)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
207,50.00000,70.00000,1.01000,0.00000,0.00000,NaN,normal,notpresent,notpresent,230.00000,50.00000,2.20000,nan,nan,12.00000,41,10400,4.6,yes,yes,no,good,no,no,1
265,50.00000,80.00000,1.02000,0.00000,0.00000,normal,normal,notpresent,notpresent,97.00000,40.00000,0.60000,150.00000,4.50000,14.20000,48,10500,5.0,no,no,no,good,no,no,0
169,nan,70.00000,1.01000,0.00000,2.00000,NaN,normal,notpresent,notpresent,220.00000,68.00000,2.80000,nan,nan,8.70000,27,NaN,NaN,yes,yes,no,good,no,yes,1
360,35.00000,60.00000,1.02500,0.00000,0.00000,normal,normal,notpresent,notpresent,105.00000,39.00000,0.50000,135.00000,3.90000,14.70000,43,5800,6.2,no,no,no,good,no,no,0
387,15.00000,80.00000,1.02500,0.00000,0.00000,normal,normal,notpresent,notpresent,93.00000,17.00000,0.90000,136.00000,3.90000,16.70000,50,6200,5.2,no,no,no,good,no,no,0


In [128]:
bank_dataset = pd.read_csv('../data/banknote/data_banknote_authentication.txt',sep=',')

In [129]:
bank_dataset.sample(5)

,variance,skewness,curtosis,entropy,class
1037,-1.79760,-6.76860,6.67530,0.89912,1
1346,1.25720,4.87310,-5.28610,-5.87410,1
717,2.98560,7.26730,-0.40900,-2.24310,0
666,1.22620,0.89599,5.75680,-0.11596,0
599,3.92920,-2.91560,2.21290,0.30817,0


In [131]:
clean_noisy_data(bank_dataset).sample(5)

,variance,skewness,curtosis,entropy,class
719,1.77480,-0.76978,5.58540,1.30390,0
552,3.68940,9.88700,-4.07880,-4.36640,0
805,-3.60850,3.32530,-0.51954,-3.57370,1
203,4.17570,10.26150,-3.85520,-4.30560,0
434,5.62720,10.08570,-4.29310,-3.81420,0
